## 아마존 데이터 합-체!!!

In [2]:
import re
import os
import nltk
import string
import gensim
import pyLDAvis
import numpy as np
import contractions
import pandas as pd
from tqdm import tqdm
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pyLDAvis.gensim as gensimvis
from matplotlib import pyplot as plt
from gensim import corpora, models
from gensim.models import doc2vec
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.doc2vec import TaggedDocument
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None) # Pandas 전체 열 확인
pd.set_option('display.max_rows', None) # Pandas 전체 행 확인

In [3]:
df_air = pd.read_csv("./amz_air_전처리.csv") # 로드할 데이터의 (상대 or 절대) 경로를 지정
df_refri = pd.read_csv("./amz_refri_전처리.csv") # 로드할 데이터의 (상대 or 절대) 경로를 지정
df_TV = pd.read_csv("./amz_TV_전처리.csv") # 로드할 데이터의 (상대 or 절대) 경로를 지정
df_wash = pd.read_csv("./amz_wash_전처리.csv") # 로드할 데이터의 (상대 or 절대) 경로를 지정

In [5]:
df_combined = pd.concat([df_air, df_refri, df_TV, df_wash], ignore_index=True)
df_selected = df_combined[['text', 'star']]
df_selected

,text,star
0,Using Fake gas & copper pipe:\r\nThe service c...,1
1,"Cooling, efficiency, everything is good but th...",3
2,Product is good but installing this thing is n...,3
3,Good,3
4,"The product is really good and nice cooling, l...",3
5,"Overall its a good product, but the ocean blac...",3
6,Purchased the AC last week.Installation was a ...,3
7,Product Quality is good but service needs to b...,3
8,"Value for money, features, cooling, performanc...",3
9,Liked very quick delivery .\r\nDislike MRP as ...,2


In [ ]:
df_selected.to_csv("selected_columns.csv", index=False)
